In [26]:
def ipython_info():
	import sys
	ip = False
	if 'ipykernel' in sys.modules:
		ip = True  # 'notebook'
	# elif 'IPython' in sys.modules:
	#    ip = 'terminal'
	return ip


ISIPYTHON = ipython_info()

In [29]:
import requests
import time

import json

from datetime import datetime



def logwrite(logfile, txt):
	timeStr = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
	msg = '[%s] : %s \n' %(timeStr, txt) 
	logfile.write(msg)


class API:
	_base_api_path = "http://theconsidition.se/considition/ironman/"
	_api_key = ""
	_max_players = 1
	_map = "standardmap"
	_number_of_elevations = 10
	_number_of_waterstreams = 10
	_number_of_powerups = 10

	def __init__(self, apiKey, maxPlayers, map, numberOfElevations, numberOfWaterstreams, numberOfPowerups, logfile):
		self._api_key = apiKey
		self._max_players = maxPlayers
		self._map = map
		self._number_of_elevations = numberOfElevations
		self._number_of_waterstreams = numberOfWaterstreams
		self._number_of_powerups = numberOfPowerups
		self._logfile= logfile

	#Gets the gamestate for a given game_id.
	#Returns: The gamestate as JSON
	def get_game(self, game_id):
		logwrite(self._logfile,"Getting game: " + game_id)
		r = requests.get(self._base_api_path+"games/"+game_id+'/'+self._api_key)
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			return response

	#Creates a new game, with a specified number of max players,
	#waterstreams, elevations, and powerups.
	#Returns: The game_id of the new game
	def init_game(self):
		r = requests.post(self._base_api_path+"games", json = { "ApiKey" :self._api_key, "MaxPlayers" : self._max_players, "Map" : self._map, "NumberOfStreams" : self._number_of_waterstreams, "NumberOfElevations" : self._number_of_elevations, "NumberOfPowerups" : self._number_of_powerups})
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			game_id = response["gameId"]
			logwrite(self._logfile,"Created new game: " + game_id)
			return game_id

	#Joins a game
	#Returns: The gamestate after the new player has been inserted
	def join_game(self, game_id):
		r = requests.post(self._base_api_path+"games/"+game_id+"/join", json = { "ApiKey" : self._api_key })
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			logwrite(self._logfile,"Joined game: " + response["gameState"]["gameId"])
			return response

	#Readies up for a game
	#Returns: The gamestate after the player has readied up
	def ready_up(self, game_id):
		logwrite(self._logfile,"Readying up!")
		r = requests.post(self._base_api_path+"games/"+game_id+"/ready", json = { "ApiKey" : self._api_key})
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			return response

	#Creates, joins, and readies up for a game
	#Returns: The ID of the started game
	def initiate_one_player_game(self):
		game_id = self.init_game()
		joined_response = self.join_game(game_id)
		readied_response = self.ready_up(game_id)
		return readied_response["gameState"]["gameId"]

	#Continously try to ready up for a game
	#To be used when joining games with more than one player
	#Returns: The gamestate after all of the players have successfully readied up
	def try_ready_for_game(self, game_id):
		logwrite(self._logfile,'Readying up!');    
		readied_response = self.ready_up(game_id)
		while (readied_response == None):
			logwrite(self._logfile,"Trying to ready up")
			time.sleep(2)
			readied_response = self.ready_up(game_id)
		return readied_response

	#Makes a move in a given direction with a given speed
	#Returns: The updated gamestate
	def make_move(self, game_id, direction, speed):
		logwrite(self._logfile,"Attempting to makeMove with speed: " + speed + " and direction: " + direction)
		r = requests.post(self._base_api_path+"games/"+game_id+"/action/move", json = { "ApiKey" : self._api_key, "Type" : "move", "Speed" : speed, "Direction" : direction})
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			return response

	#Takes a step in a given direction
	#Returns: The updated gamestate
	def step(self, game_id, direction):
		logwrite(self._logfile,"Attempting to step in direction: " + direction)
		r = requests.post(self._base_api_path+"games/"+game_id+"/action/step", json = { "ApiKey" : self._api_key, "Direction" : direction})
		if (r != None):
			response = r.json()
			if (response["success"] != True):
				logwrite(self._logfile,response["message"])
			else:
				return response

	#Rests for 1 turn
	#Returns: The updated gamestate
	def rest(self, game_id):
		logwrite(self._logfile,"Attempting to rest!")
		r = requests.post(self._base_api_path + "games/" + game_id + "/action/rest", json={"ApiKey": self._api_key})
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			return response

	#Uses a chosen powerup
	def use_powerup(self, game_id, powerup_name):
		logwrite(self._logfile,"Attempting to use powerup: " + powerup_name)
		r = requests.post(self._base_api_path + "games/" + game_id + "/action/usepowerup", json={"ApiKey": self._api_key, "Name" : powerup_name})
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			return response

	#Drops a chosen powerup
	def drop_powerup(self, game_id, powerup_name):
		logwrite(self._logfile,"Attempting to drop powerup: " + powerup_name)
		r = requests.post(self._base_api_path + "games/" + game_id + "/action/droppowerup", json={"ApiKey": self._api_key, "Name" : powerup_name})
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])
		else:
			return response

	#Ends previous active games
	def end_previous_games_if_any(self):
		logwrite(self._logfile,"Attempting to end previous games if any.")
		r = requests.delete(self._base_api_path + "games", json={"ApiKey": self._api_key})
		response = r.json()
		if (response["success"] != True):
			logwrite(self._logfile,response["message"])


In [30]:
direction_types = {
    'w': 400,
    's': 500,
    'e': 600,
    'n': 700,
    'aoe': 800
}

terrain_types = {'road': 1000,
                 'trail': 1100,
                 'grass': 1200,
                 'water': 1300,
                 'rockywater': 1400,
                 'forest': 1500,
                 'start': 1600,
                 'win': 1700
                 }

terrain_dynamics = {'rain': 10000,
                    'elevation': 20000,
                    'waterstream': 30000,
                    'powerup': 40000}

powerup_types = dict(
    RemoveCloud=1010015,
    RestoreStamina=1100011,
    InvertStreams=1030012,
    Shoes=2201113,
    Flippers=2201313,
    Cycletire=2201013,
    Umbrella=2110014,
    Energyboost=2100015,
    Potion=2200016,
    Helmet=2300017,
    StaminaSale=2100018,
    Spikeshoes=2401119,
    Cyklop=2401319,
    BicycleHandlebar=2401019
)

import json
import numpy as np
from pprint import pprint as ppr


def mapdata_parser(path, stateFname='state', mapdataFname='mapdata', doOnlyDyn=True):
    with open('%s/%s.json' % (path, stateFname), 'r') as statefile:
        state = json.load(statefile)
        tileInfo = state['tileInfo']
        mapTerrain = np.zeros([100, 100])
        mapDynamics = np.zeros([100, 100])

        for i, row in enumerate(tileInfo):
            for j, col in enumerate(row):
                terrainType = terrain_types[col.pop('type')]
                mapTerrain[i, j] = terrainType
                if len(col):
                    for k in col.keys():
                        v = 0
                        if k == 'elevation':
                            v = col[k]
                            amount = v['amount']
                            direction = direction_types[v['direction']]
                            dynamics = terrain_dynamics[k]
                            v = dynamics + direction + amount
                        elif k == 'waterstream':
                            v = col[k]
                            speed = v['speed']
                            direction = direction_types[v['direction']]
                            dynamics = terrain_dynamics[k]
                            v = dynamics + direction + speed
                        elif k == 'weather':
                            v = terrain_dynamics[col[k]]
                        elif k == 'powerup':
                            v = powerup_types[col[k]['name']]
                        mapDynamics[i, j] = v

    with open('%s/%s.dynamic.csv' % (path, mapdataFname), 'w') as mapdatafile:
        for i in range(100):
            for j in range(100):
                if mapDynamics[i, j]:
                    mapdatafile.write("%d,%d,%d\n" % (i, j, mapDynamics[i, j]))

    if not doOnlyDyn:
        with open('%s/%s.static.csv' % (path, mapdataFname), 'w') as mapdatafile:
            for i in range(100):
                for j in range(100):
                    mapdatafile.write("%d\n" % (mapTerrain[i, j]))


if __name__ == '__main__':
    mapdata_parser(path='../datadir', stateFname='state', mapdataFname='mapdata')


In [ ]:
from IPython.core.display import display, HTML
from IPython.display import IFrame
display(HTML("<style>.container { width:100% !important; }</style>"))


gameIdFile="gameid.txt"
w=1100
h=500
gameIdFile = open(gameIdFile,'r')
gameId = gameIdFile.read()
gameIdFile.close()
IFrame(src="http://www.theconsidition.se/ironmanvisualizer?gameId=%s"%gameId, width=w, height=h)

In [ ]:
def ipython_info():
	import sys
	ip = False
	if 'ipykernel' in sys.modules:
		ip = True  # 'notebook'
	# elif 'IPython' in sys.modules:
	#    ip = 'terminal'
	return ip


ISIPYTHON = ipython_info()
import json

from datetime import datetime



def logwrite(logfile, txt):
	timeStr = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
	msg = '[%s] : %s \n' %(timeStr, txt) 
	logfile.write(msg)

import random
import sys

if not ISIPYTHON:
    from api import API
    from draw_map import draw_map
    from mapdata_parser import mapdata_parser

from pprint import pprint as ppr

import time

#A "solution" that takes a step in a random direction every turn
def solution(game_id, logfile, stateFname, mapdataFname, path):
	gotMap = False
	initial_state = _api.get_game(game_id)
	if (initial_state["success"] == True):
		state = initial_state["gameState"]
		tiles = state["tileInfo"]
		current_player = state["yourPlayer"]
		current_y_pos = current_player["yPos"]
		current_x_pos = current_player["xPos"]
		while not state["gameStatus"] == "done":
			t1 = time.time()
			logwrite(logfile,"Starting turn: " + str(state["turn"]) )
			tiles = state["tileInfo"]
			logwrite(logfile,str(state["yourPlayer"]))
			current_player = state["yourPlayer"]
			current_y_pos = current_player["yPos"]
			current_x_pos = current_player["xPos"]
			# Take a step in a random direction
			step_direction_array = ["w", "e", "n", "s"]
			random_step = random.randint(0, 3)
			logwrite(logfile,"Stepped: " + str(step_direction_array[random_step]))
			response = _api.step(game_id, step_direction_array[random_step])
			state = response["gameState"]

			with open('%s/%s.json'%(path, stateFname), 'w') as outfile:
				json.dump(state, outfile)

			t2 = time.time()

			if gotMap == False:
				gotMap = True
				mapdata_parser(path, stateFname, mapdataFname, doOnlyDyn=True)
			else:
				mapdata_parser(path, stateFname, mapdataFname, doOnlyDyn=False)
                
			t3 = time.time()

			print("Reading data time: %f, Parsing data time: %f"%(t2-t1, t3-t1))

            
            
			#draw_map(state)
			
		logwrite(logfile,"Finished!")
	else:
		logwrite(logfile,initial_state["message"])


gameId = None



def ironman(_api, path, stateFname, mapdataFname, logfile, gameId = None):

	logfile.write ("start ironman with gameId : %s"% str(gameId))
	#If no gameID is specified as parameter to the script,
	#Initiate a game with 1 player on the standard map
	if (gameId is None):
		_api.end_previous_games_if_any() #Can only have 2 active games at once. This will end any previous ones.
		gameId = _api.init_game()
		gameIdFile = open("gameid.txt",'w')
		gameIdFile.write(gameId)
		gameIdFile.close()
		ironman(_api, path, stateFname, mapdataFname, logfile, gameId)
	else:
		joinedGame = _api.join_game(gameId)
		readiedGame = _api.try_ready_for_game(gameId)
		if (readiedGame != None):
			logwrite(logfile,"Joined and readied! Solving...")
			solution(gameId,logfile, stateFname, mapdataFname, path)


if __name__ == "__main__":
	path = '../datadir'
	logFname = 'ironman.log'
	mapdataFname = 'mapdata'
	stateFname = 'statedata'


	# Insert your API-key here
	_api_key = "326dcc2e-6ccc-457a-bd12-596eadc87a64"
	#Specify your API-key number of players per game),
	# mapname, and number of waterstreams/elevations/powerups here

	with open("%s/%s.txt"%(path,logFname),"w") as logfile:
		_api = API(_api_key, 1, "standardmap", 10, 10, 10, logfile)
		ironman(_api, path, stateFname, mapdataFname, logfile, )
